In [1]:
import geemap
import folium
import ee
import os
import cv2
import numpy as np
from PIL import Image
import numpy as np
import palettable 
import numpy as np
from Satellite import *
from Simulator import *
service_account = "jims-948@jimssatellites.iam.gserviceaccount.com"
key_path = 'jimssatellites-25b0240d5cf2.json'
credentials = ee.ServiceAccountCredentials(service_account, key_path)
ee.Initialize(credentials=credentials)



count = 0
import csv
sat1 = Satellite("Satellites/sat1.txt", "SAT1")
sat2 = Satellite("Satellites/sat2.txt", "SAT2")
sat3 = Satellite("Satellites/sat3.txt", "SAT3")
sat4 = Satellite("Satellites/sat4.txt", "SAT4")
sat5 = Satellite("Satellites/sat5.txt", "SAT5")
sat6 = Satellite("Satellites/sat6.txt", "SAT6")  
sat7 = Satellite("Satellites/sat7.txt", "SAT7")
sat8 = Satellite("Satellites/sat8.txt", "SAT8")
sat9 = Satellite("Satellites/sat9.txt", "SAT9")  
sat10 = Satellite("Satellites/sat10.txt", "SAT10")
sat11 = Satellite("Satellites/sat11.txt", "SAT11")
sat12 = Satellite("Satellites/sat12.txt", "SAT12")

satellites = [sat1,sat2,sat3,sat4,sat5,sat6,sat7,sat8,sat9,sat10,sat11,sat12]

count = 0
mappingArray = []
for sat in satellites:
    csv_file = "RESULTS_"+sat.name+".csv"
    count 
    with open(csv_file, mode='r', newline='') as file:
        reader = csv.reader(file)  
        i = 0
        for row in reader:
            coord1a, coord1b,  coord4a, coord4b, error = [float(param) for param in row]
            roi = ee.Geometry.Rectangle([[coord1a, coord1b],  [coord4a, coord4b]])
            newFeature = ee.Feature(roi, {'name': str(count)})
            count = count+1
            mappingArray.append(newFeature)
            count+=1




In [2]:

# Create an interactive map
Map = geemap.Map(center=[0, 0], zoom=2)
mappedArea = ee.FeatureCollection(mappingArray)
NSW = ee.Geometry.Rectangle([[144.81577841465915, -36.42295520530404], [155.81727106335805, -27.806830550620564]])
# Create an image collection for Sentinel-2 data
start_date = '2019-06-01'
end_date = '2020-05-31'
sentinel2_collection = (ee.ImageCollection('COPERNICUS/S2_SR')
                        .filterDate(ee.Date(start_date), ee.Date(end_date))
                        .filterBounds(mappedArea)
                        .select('B.*|MSK_CLDPRB|SCL')
)
burnedArea = ee.Image('projects/jimssatellites/assets/fires')
nir = sentinel2_collection.select('B8')  # Near-Infrared Band (B8)
swir1 = sentinel2_collection.select('B11')  # Short-Wave Infrared Band 1 (B11)
swir2 = sentinel2_collection.select('B12')  # Short-Wave Infrared Band 2 (B12)
##CREATE THE BLACK MASK

roi = ee.Geometry.Rectangle([ 146,-35.36,  158.6, -30.83])
#I have no idea why, but for some reason this just paints everything so I'm gonna cop for now
mask = ee.Image(0).toInt8().paint(roi, 1)
fc = ee.FeatureCollection(mappingArray)
# Fill and outline the polygons in two colors
filled = mask.paint(fc, 2)
both = filled.paint(fc, 1, 5)
# Mask off everything that matches the fill color.
result = both.mask(filled.neq(2))


nir_palette = ['000000', '330000', '660000', '990000', 'CC0000', 'FF0000', 'FF1900', 'FF3200', 'FF4B00', 'FF6400',
              'FF7D00', 'FF9600', 'FFAF00', 'FFC800', 'FFE100', 'FFFF00']
# Create a visualization palette
vis_params = {
    'min': 0,
    'max': 10000,
    'bands': ['B11']
}
# Define visualization parameters for each band
nir_vis_params = {
    'min': 0,
    'max': 10000,
    'bands': 'B11',
    'palette': nir_palette  # Apply the custom color palette for NIR
}
swir_vis_params = {
    'min': 0,
    'max': 10000,
    'bands': 'B12',
    'palette': nir_palette  # Apply the custom color palette for NIR
}

#Map.addLayer(sentinel2_collection, vis_params, 'Sentinel-2 Image')
#Map.addLayer(nir, nir_vis_params, 'NIR')



#MOSAICING
swir1_mosaic = swir1.mosaic()
swir1_mosaic = swir1_mosaic.visualize(**{'bands': 'B11', 'min': 0,'max': 10000, 'palette': nir_palette})
mask = result.visualize(**{'palette': '000000,FF0000','max': 10000,'opacity': 1.0})
mosaic = ee.ImageCollection([swir1_mosaic, mask]).mosaic()


#Add the historic data to seperate mosaic
burnedVisParams = {
    'min': 0,
    'max': 255,
    'palette': [  'fcffa4', 'f2f482', 'f3e35a', 'f8cf3a', 'fbbc21', 'fca80d', 'fb9606',
  'f8850f', 'f2741c', 'eb6429', 'e15635', 'd54a41', 'c83f4b', 'bc3754',
  'ad305d', '9d2964', '8d2369', '7d1e6d', '6d186e', '5d126e', '4f0d6c',
  '3e0966', '2d0b59', '1c0c43', '0e092b', '040314', '000004']
}

roi = ee.Geometry.Rectangle([ 146,-35.36,  158.6, -30.83], None, False)
burnedAreaViz = burnedArea.visualize(**burnedVisParams)
historic_mosaic = ee.ImageCollection([swir1_mosaic, burnedAreaViz, mask]).mosaic()



In [4]:
from PIL import Image as im 

def getDangerRating(ratio):
    if ratio>= 0 and ratio<= 5:
        return "MODERATE"
    elif ratio>5 and ratio<= 15:
        return "HIGH"
    elif ratio>15 and ratio<25:
        return "EXTREME"
    else:
        return "CATASTROPHIC"




current_directory = os.getcwd()
out_dir = os.path.join(current_directory)

#Check Comleroy
comleroy = [  150.78448429,-33.38,  151.2087900435, -33.0931176]
#Save mapped mosaic
filename = os.path.join(out_dir, 'eeResults/comleroy.tif')
roi = ee.Geometry.Rectangle(comleroy, None, False)
image = mosaic.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=50, region=roi, file_per_band=False
)

#Save mapped mosaic with historical data
filename = os.path.join(out_dir, 'eeResults/comleroy_historic.tif')
roi = ee.Geometry.Rectangle(comleroy, None, False)
image = historic_mosaic.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=50, region=roi, file_per_band=False
)


# Open the TIFF file
tiff_file = "eeResults/comleroy.tif"
image = Image.open(tiff_file)
image = np.array(image)
#Convert to jpg
cv2.imwrite("eeResults/comleroy.jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
# Apply filter
lower1 = np.array([50, 0, 0])
upper1 = np.array([90, 2, 2])
mask = cv2.inRange(image, lower1, upper1)
white = 0
total = 0
for row in mask:
    for pixel in row:
        if (pixel == 255):
            white+=1
        total+=1
ratio =white/total * 100
print("Comleroy danger percentage:",ratio)



#Check Yaouk
yaouk = [148.7428497953762, -35.96021682698345, 149.0150704273652, -35.71831894036284]
filename = os.path.join(out_dir, 'eeResults/yaouk.tif')
roi = ee.Geometry.Rectangle(yaouk, None, False)
image = mosaic.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=50, region=roi, file_per_band=False
)
filename = os.path.join(out_dir, 'eeResults/yaouk_historic.tif')
roi = ee.Geometry.Rectangle(yaouk, None, False)
image = historic_mosaic.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=50, region=roi, file_per_band=False
)


# Open the TIFF file
tiff_file = "eeResults/yaouk.tif"
image = Image.open(tiff_file)
image = np.array(image)
#Convert to jpg
cv2.imwrite("eeResults/yaouk.jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
# Apply filter
lower1 = np.array([50, 0, 0])
upper1 = np.array([90, 2, 2])
mask = cv2.inRange(image, lower1, upper1)
white = 0
total = 0
for row in mask:
    for pixel in row:
        if (pixel == 255):
            white+=1
        total+=1
ratio =white/total * 100
print("Yaouk danger percentage:",ratio)


#Check piliga
piliga =[148.99253089357404,-31.069826237629815, 149.57756512477508, -30.659188247907124]
filename = os.path.join(out_dir, 'eeResults/piliga.tif')
roi = ee.Geometry.Rectangle(piliga, None, False)
image = mosaic.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=50, region=roi, file_per_band=False
)
filename = os.path.join(out_dir, 'eeResults/piliga_historic.tif')
roi = ee.Geometry.Rectangle(piliga, None, False)
image = historic_mosaic.clip(roi).unmask()
geemap.ee_export_image(
    image, filename=filename, scale=50, region=roi, file_per_band=False
)
# Open the TIFF file
tiff_file = "eeResults/piliga.tif"
image = Image.open(tiff_file)
image = np.array(image)
#Convert to jpg
cv2.imwrite("eeResults/piliga.jpg", cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
# Apply filter
lower1 = np.array([50, 0, 0])
upper1 = np.array([90, 2, 2])
mask = cv2.inRange(image, lower1, upper1)
white = 0
total = 0
for row in mask:
    for pixel in row:
        if (pixel == 255):
            white+=1
        total+=1
ratio =white/total * 100
print("Piliga danger percentage:",ratio)

Map.addLayer(mosaic, {}, 'Mosaic')
Map.addLayer(historic_mosaic, {}, "historic_mosaic")
Map


Generating URL ...
Please wait ...
Data downloaded to c:\Users\Tom\Documents\Uni\AERO3760\Jims_Satellites\Simulator 2\eeResults\comleroy.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Tom\Documents\Uni\AERO3760\Jims_Satellites\Simulator 2\eeResults\comleroy_historic.tif
Comleroy danger percentage: 16.10105074893807
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Tom\Documents\Uni\AERO3760\Jims_Satellites\Simulator 2\eeResults\yaouk.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Tom\Documents\Uni\AERO3760\Jims_Satellites\Simulator 2\eeResults\yaouk_historic.tif
Yaouk danger percentage: 32.2539508206724
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Tom\Documents\Uni\AERO3760\Jims_Satellites\Simulator 2\eeResults\piliga.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\Tom\Documents\Uni\AERO3760\Jims_Satellites\Simulator 2\eeResults\piliga_historic.tif
Piliga danger percentage: 1.0627849141745196


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…